In [1]:
import pandas as pd
from tsnn.data_utils import *
from tsnn.models import LSTNet

Using TensorFlow backend.


In [2]:
batch_size = 64
timesteps = 168
sampling_step = 1
pred_delay = 24
pred_length = 1
nb_input_features = 321
input_cols = list(range(321))
interest_vars = [315]

In [3]:
# Load data
raw_data = pd.read_csv('electricity.txt', header=None)

# Scale data
scaled_data, stats_df = scale_maxabs(raw_data)

# Split into inputs and targets
inputs, targets = inputs_targets_split(scaled_data, 
                                       input_cols=input_cols, 
                                       target_cols=interest_vars,
                                       samples_length=timesteps, 
                                       pred_delay=pred_delay, 
                                       pred_length=pred_length)

# Train-Val-Test split
train_idx, val_idx, test_idx = train_val_split(targets, train_ratio=0.6, val_ratio=0.2)

In [4]:
# Create model
model = LSTNet(input_shape=(timesteps, nb_input_features), 
               interest_vars=interest_vars)
model.compile(loss='mae', optimizer='adam')

In [5]:
# Prepare data generators
train_gen = sample_gen_rnn(inputs, targets, train_idx, timesteps, sampling_step, batch_size)
val_gen = sample_gen_rnn(inputs, targets, val_idx, timesteps, sampling_step, batch_size)
tr_steps_per_epoch = (train_idx[1] - train_idx[0] + 1) // batch_size + 1
val_steps_per_epoch = (train_idx[1] - train_idx[0] + 1) // batch_size + 1

# Fit model
model.fit_generator(generator=train_gen, 
                    steps_per_epoch=tr_steps_per_epoch, 
                    validation_data=val_gen, 
                    validation_steps=val_steps_per_epoch,
                    epochs=50, 
                    shuffle=False)

Epoch 1/50
245/245 [==============================] - 176s - loss: 0.2457 - val_loss: 0.1169

KeyboardInterrupt: 